# NN Chatbot Training 

### Trains the conversational attributes of our chatbot
<hr>

<br>

# Imports
<hr>

In [40]:
# Import

import json 
import numpy as np 

import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Var Declaration

In [41]:
# 
training_sentences = []
training_labels = []
labels = []
responses = []
#
vocab_size = 1000
embedding_dim = 16
max_len = 20 
oov_token = '<00V>'

<br>

# Data Ingestion
<hr>

In [42]:
"""
    This code block opens the json file and enters the data into arrays (lists),
    so that it can be feed into the pipeline. 

"""

# Open json file
with open('../data/intents.json') as file:
    data = json.load(file)



# loop to load data 
for intent in data['intents']:
    for pattern in intent['patterns']:
        #
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    #
    responses.append(intent['responses'])

    # creates labels list
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


num_classes = len(labels)

In [43]:
# inspect Data

print(f'Training Sentences: {training_sentences[:7]}')
print(f'Training Labels:    {training_labels[:6]}')
print(f'Labels:             {labels[:7]}')
print(f'Responses:          {responses[:3]}')

Training Sentences: ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later']
Training Labels:    ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye']
Labels:             ['greeting', 'goodbye', 'thanks', 'about', 'name', 'help', 'bookaroom']
Responses:          [['Hello', 'Hi', 'Hi there'], ['See you later', 'Have a nice day', 'Bye! Come back again'], ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]]


<br> 

# Preprocessing
<hr>

##### Normalize Labels w/LabelEncoder 

In [44]:

# define model 
lbl_encoder = LabelEncoder()
# fit model 
lbl_encoder.fit(training_labels)
# transform model 
training_labels = lbl_encoder.transform(training_labels)



In [45]:
# View Encoded Labels

training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 1, 1, 1, 1, 1, 2, 2, 2])


##### Normalize Tokenizer Data

In [46]:
"""
 Tokenize data
 
 setting number of words to 1,000
 Using oov_token 

"""

# declare tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
# fit tokenier
tokenizer.fit_on_texts(training_sentences)
# creates dictonary of words
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
# creates equal sizes between the text sequences
padded_sequences = pad_sequences(sequences, truncating = 'post', maxlen = max_len)


In [47]:
# inspect tokenized var

print()
print(f'Len of word_index: {len(word_index)}')
print(f'Sequences:         {sequences[:7]} ')
print()
print('padded Sequences: ')
padded_sequences[:3]


Len of word_index: 54
Sequences:         [[27], [28], [9, 29, 16], [30], [31], [32], [33, 2, 34]] 

padded Sequences: 


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 27],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 28],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  9, 29, 16]], dtype=int32)

In [48]:
# define model 

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation = 'softmax'))


In [49]:
# Compile model

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [50]:
# Model Summory 

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_9 (Dense)             (None, 16)                272       
                                                                 
 dense_10 (Dense)            (None, 16)                272       
                                                                 
 dense_11 (Dense)            (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [51]:
# Train Model 

epochs = 555
history = model.fit(padded_sequences, np.array(training_labels),  epochs=epochs)

Epoch 1/555
2/2 [==============================] - 1s 7ms/step - loss: 2.0799 - accuracy: 0.1212
Epoch 2/555
2/2 [==============================] - 0s 10ms/step - loss: 2.0787 - accuracy: 0.2121
Epoch 3/555
2/2 [==============================] - 0s 14ms/step - loss: 2.0779 - accuracy: 0.2121
Epoch 4/555
2/2 [==============================] - 0s 33ms/step - loss: 2.0770 - accuracy: 0.2121
Epoch 5/555
2/2 [==============================] - 0s 8ms/step - loss: 2.0762 - accuracy: 0.2121
Epoch 6/555
2/2 [==============================] - 0s 9ms/step - loss: 2.0756 - accuracy: 0.2121
Epoch 7/555
2/2 [==============================] - 0s 6ms/step - loss: 2.0750 - accuracy: 0.2121
Epoch 8/555
2/2 [==============================] - 0s 7ms/step - loss: 2.0743 - accuracy: 0.2121
Epoch 9/555
2/2 [==============================] - 0s 6ms/step - loss: 2.0735 - accuracy: 0.2121
Epoch 10/555
2/2 [==============================] - 0s 11ms/step - loss: 2.0727 - accuracy: 0.2121
Epoch 11/555
2/2 [=======

# Making a Pickle
<hr>

In [52]:
# save the trained model 
model.save('../assets/chat_model')

# save the fitted Tokenizer
with open('../assets/tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fittend label encoder
with open('../assets/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


    

INFO:tensorflow:Assets written to: ../assets/chat_model/assets
